In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [2]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

from dotenv import load_dotenv
_ = load_dotenv()

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [5]:
api_key = os.environ["AZURE_OPENAI_API_KEY"]
llm_model = "gpt-4o"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
    api_base="https://shav.openai.azure.com",
    api_version="2023-03-15-preview"
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [6]:
# parquet files generated from indexing pipeline
INPUT_DIR = "../bin/output/20240902-205939/artifacts" 
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [7]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

Z:\code\007\graphrag-pipeline\graphrag\query\indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
Z:\code\007\graphrag-pipeline\graphrag\query\indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


Total report count: 297
Report count after filtering by community level 2: 230


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,293,# MVP and Pre-Seed Investors in Startup Ecosys...,3,9.0,MVP and Pre-Seed Investors in Startup Ecosystems,The rating is high due to the critical role of...,The community revolves around the development ...,[{'explanation': 'The Minimum Viable Product (...,"{\n ""title"": ""MVP and Pre-Seed Investors in...",219856f2-57e0-4290-b752-b4e9f46cd353
1,294,# Early Adopters and Marketing Strategy in Sta...,3,9.0,Early Adopters and Marketing Strategy in Start...,The rating is high due to the significant impa...,The community revolves around the critical rol...,[{'explanation': 'Early adopters are pivotal i...,"{\n ""title"": ""Early Adopters and Marketing ...",9dfe4be6-bb1d-4730-9c7d-b54f0a716c52
2,295,# Pricing Strategy and Competitive Landscape i...,3,9.0,Pricing Strategy and Competitive Landscape in ...,The rating is high due to the significant impa...,The community revolves around the critical ele...,[{'explanation': 'Pricing strategy is a fundam...,"{\n ""title"": ""Pricing Strategy and Competit...",0bfa1f66-78b1-4e36-b3b9-105ee2fece42
3,296,# Subscription Revenue Model and Recurring Rev...,3,9.0,Subscription Revenue Model and Recurring Revenue,The rating is high due to the significant impa...,The community revolves around the Subscription...,[{'explanation': 'The Subscription Revenue Mod...,"{\n ""title"": ""Subscription Revenue Model an...",a9abac19-f949-43a9-9793-ba267d1a1902
4,159,# AXE and #PraiseUp Campaign\n\nThe community ...,2,8.5,AXE and #PraiseUp Campaign,The rating is high due to the significant impa...,The community revolves around AXE and its #Pra...,[{'explanation': 'AXE has demonstrated a keen ...,"{\n ""title"": ""AXE and #PraiseUp Campaign"",\...",ddb037ff-05af-4278-aa04-3ac77d0c50ea


#### Build global context based on community reports

In [8]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [9]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [10]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [11]:
result = await search_engine.asearch(
    "What is the major lessos for Entrepreneurs?"
)

print(result.response)

## Major Lessons for Entrepreneurs

Entrepreneurs can draw numerous critical lessons from the experiences of successful and failed startups. These lessons span various aspects of the entrepreneurial journey, from problem identification to securing funding and achieving market fit. Below are some of the most significant insights:

### Importance of a Well-Defined Problem Statement

A well-defined problem statement is foundational for identifying market gaps and guiding product development. Successful companies like Uber, Slack, and Robinhood have demonstrated the importance of clearly understanding and articulating the problem they aim to solve [Data: Reports (248)].

### Securing Financial Backing

Securing financial backing is crucial for startups to scale and innovate. Early-stage funding from angel investors and venture capitalists (VCs) is essential for developing products, iterating the Minimum Viable Product (MVP), and making critical hires. Vloggi's pursuit of seed funding exemp

In [12]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,188,Market Dynamics and Key Influencers in Entrepr...,0.506024,# Market Dynamics and Key Influencers in Entre...,9.0
1,225,Revenue and Business Models in Startups,0.445783,# Revenue and Business Models in Startups\n\nT...,9.0
2,73,Customer Acquisition and Sentiment in Entrepre...,0.253012,# Customer Acquisition and Sentiment in Entrep...,9.0
3,137,Economic Moats and Competitive Advantages in S...,0.216867,# Economic Moats and Competitive Advantages in...,9.0
4,238,Market Research and Entrepreneurial Ecosystem,0.120482,# Market Research and Entrepreneurial Ecosyste...,9.0
...,...,...,...,...,...
225,236,Freewrite and Distraction-Free Typing,0.024096,# Freewrite and Distraction-Free Typing\n\nThe...,7.5
226,54,Partnerships and Multinational Companies in St...,0.012048,# Partnerships and Multinational Companies in ...,8.5
227,240,Upwork and Designer Ecosystem,0.012048,# Upwork and Designer Ecosystem\n\nThe communi...,8.5
228,109,Product Ideas and Elevator Innovations,0.012048,# Product Ideas and Elevator Innovations\n\nTh...,8.5


In [13]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 14. LLM tokens: 169852
